# Processing structures

In [1]:
import os

import pandas as pd
from Bio.PDB.PDBExceptions import PDBConstructionException
from Bio.SeqUtils import IUPACData
from stcrpy import fetch_TCR

In [2]:
stcrdab_summary = pd.read_csv('data/raw/stcrdab_summary.tsv', delimiter='\t')
stcrdab_summary['resolution'] = pd.to_numeric(stcrdab_summary['resolution'], errors='coerce')

print('Number of structures', len(stcrdab_summary))

Number of structures 984


In [3]:
selected_stcrdab = stcrdab_summary.query(
    "TCRtype == 'abTCR' and resolution <= 3.5 and antigen_type == 'peptide' and mhc_type in ('MH1', 'MH2')"
).copy()

print('Number of structures', len(selected_stcrdab))

Number of structures 436


In [4]:
output_dir = 'data/interim/structures'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [11]:
selected_structures_summary = []

for pdb_id in selected_stcrdab['pdb'].unique():
    print('Loading', pdb_id)

    try:
        tcrs = fetch_TCR(pdb_id)

    except PDBConstructionException as err:
        print('Skipping', pdb_id, 'because of', err)
        continue

    if not isinstance(tcrs, list):
        tcrs = [tcrs]

    for tcr in tcrs:
        tcr_info = {
            'pdb_id': pdb_id,
            'alpha_chain_id': tcr.VA,
            'beta_chain_id': tcr.VB,
        }

        for cdr in tcr.get_CDRs():
            tcr_info[cdr.id + '_sequence'] = ''.join([
                IUPACData.protein_letters_3to1[res.get_resname().title()] for res in cdr
                if res.get_resname().title() in IUPACData.protein_letters_3to1
            ])

        if len(tcr.antigen) > 1:
            print('Discarding multi-antigen')
            continue

        try:
            tcr_info['antigen_chain_id'] = tcr.antigen[0].id
        except IndexError as err:
                print('Skipping', pdb_id, 'becasue of', err)
                continue

        tcr_info['antigen_sequence'] = ''.join([
            IUPACData.protein_letters_3to1[res.get_resname().title()] for res in tcr.antigen[0]
            if res.get_resname().title() in IUPACData.protein_letters_3to1
        ])

        try:
            entry_sequences = (
                tcr_info['cdra1_sequence'],
                tcr_info['cdra2_sequence'],
                tcr_info['cdra3_sequence'],
                tcr_info['cdrb1_sequence'],
                tcr_info['cdrb2_sequence'],
                tcr_info['cdrb3_sequence'],
                tcr_info['antigen_sequence'],
            )
        except KeyError as err:
            print('Skipping', pdb_id, 'becasue of', err)
            continue

        already_added = False
        for past_entry in selected_structures_summary:
            past_entry_sequences = (
                past_entry['cdra1_sequence'],
                past_entry['cdra2_sequence'],
                past_entry['cdra3_sequence'],
                past_entry['cdrb1_sequence'],
                past_entry['cdrb2_sequence'],
                past_entry['cdrb3_sequence'],
                past_entry['antigen_sequence'],
            )

            if entry_sequences == past_entry_sequences:
                already_added = True
                break

        if already_added:
            print('Skipping TCR with same CDR and antigen sequences')
            continue

        if len(tcr.MHC) > 1:
            print('Discarding multi-MHC')
            continue

        try:
            tcr_info['mhc1_chain_id'] = tcr.MHC[0].id[0]

        except IndexError as err:
            print('Skipping', pdb_id, 'because of', err)
            continue

        tcr_info['mhc2_chain_id'] = tcr.MHC[0].id[1] if len(tcr.MHC[0]) > 1 else None

        for key, info in tcr.get_germlines_and_alleles().items():
            match key:
                case 'VA':
                    tcr_info['v_alpha_gene'] = info[0]
                    tcr_info['j_alpha_gene'] = info[1]

                case 'VB':
                    tcr_info['v_beta_gene'] = info[0]
                    tcr_info['j_beta_gene'] = info[1]

                case 'VA_species':
                    tcr_info['alpha_chain_species'] = info

                case 'VB_species':
                    tcr_info['beta_chain_species'] = info

                case 'TCR_VA_seq':
                    tcr_info['alpha_chain_sequence'] = info

                case 'TCR_VB_seq':
                    tcr_info['beta_chain_sequence'] = info

                case 'MHC_GA1' | 'MHC_GA':
                    tcr_info['mhc1_gene'] = info

                case 'MHC_GA1_seq' | 'MHC_GA_seq':
                    tcr_info['mhc1_sequence'] = info

                case 'MHC_B2M' | 'MHC_GB':
                    tcr_info['mhc2_gene'] = info

                case 'MHC_B2M_seq' | 'MHC_GB_seq':
                    tcr_info['mhc2_sequence'] = info

                case 'antigen':
                    pass

                case _:
                    tcr_info[key.lower()] = info

        try:
            docking_geometry = tcr.calculate_docking_geometry(mode='cys')

        except (ValueError, ZeroDivisionError, KeyError) as err:
            print('Skipping', pdb_id, 'because of', err)
            continue

        tcr_info['pitch_angle'] = docking_geometry['pitch_angle']
        tcr_info['scanning_angle'] = docking_geometry['scanning_angle']

        tcr_name = (
            f"{pdb_id}_"
            f"{tcr_info['alpha_chain_id']}{tcr_info['beta_chain_id']}"
            f"{tcr_info['antigen_chain_id']}"
            f"{tcr_info['mhc1_chain_id']}{tcr_info['mhc2_chain_id']}"
        )

        try:
            interactions = tcr.profile_peptide_interactions()

        except KeyError as err:
            print('Skipping structure', pdb_id, 'because of', err)
            continue

        interactions.to_csv(os.path.join(output_dir, tcr_name + '_interactions.csv'), index=False)
        tcr.save(os.path.join(output_dir, tcr_name + '.pdb'))

        selected_structures_summary.append(tcr_info)

selected_structures_summary = pd.DataFrame(selected_structures_summary)
selected_structures_summary

Loading 4ozi


/ceph/project/koohylab/bmcmaste/projects/tcr-pmhc-structures-hackathon/.venv/lib/python3.11/site-packages/stcrpy/tcr_processing/TCRParser.py:480: UserWarning: Single chain MH class II instantiated with chain type GB. It is possible the other MHC class II chain has not been identified.
  warnings.warn(
Crystal Contact Warning: antigen J has been paired with TCR HG
Crystal Contact Warning: antigen I has been paired with TCR FE
Crystal Contact Warning: antigen J has been paired with TCR B
Crystal Contact Warning: antigen I has been paired with TCR D


Discarding multi-antigen
Discarding multi-antigen
Loading 2ak4


Common molecule IOD found in the binding site - not considered an antigen
Common molecule IOD found in the binding site - not considered an antigen
Common molecule IOD found in the binding site - not considered an antigen
Common molecule IOD found in the binding site - not considered an antigen


Skipping TCR with same CDR and antigen sequences
Skipping TCR with same CDR and antigen sequences
Loading 5eu6


...

/ceph/project/koohylab/bmcmaste/projects/tcr-pmhc-structures-hackathon/.venv/lib/python3.11/site-packages/stcrpy/tcr_processing/TCRParser.py:480: UserWarning: Single chain MH class II instantiated with chain type GB. It is possible the other MHC class II chain has not been identified.
  warnings.warn(


Discarding multi-antigen
Loading 4jfe


Common molecule SO4 found in the binding site - not considered an antigen
Common molecule SO4 found in the binding site - not considered an antigen


Loading 3mv9
Skipping TCR with same CDR and antigen sequences
Loading 6q3s
Skipping TCR with same CDR and antigen sequences
Loading 5men
Loading 1ao7


,pdb_id,alpha_chain_id,beta_chain_id,cdrb1_sequence,cdrb2_sequence,cdrb3_sequence,cdra1_sequence,cdra2_sequence,cdra3_sequence,antigen_chain_id,...,j_beta_gene,beta_chain_species,beta_chain_sequence,mhc_type,mhc1_gene,mhc1_sequence,mhc2_gene,mhc2_sequence,pitch_angle,scanning_angle
0,2ak4,D,E,MNHNS,SASEGT,ASPGLAGEYEQY,TRDTTYY,RNSFDEQN,ALSGFYNTDKLI,C,...,TRBJ2-7*01,[human],GVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYY...,MH1,HLA-B*3508,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,5.963917,69.096114
1,2ak4,T,U,MNHNS,SASEGT,ASPGLAEQY,TRDTTYY,RNSFDEQN,ALSGFYNTDKLI,S,...,TRBJ2-7*01,[human],GVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYY...,MH1,HLA-B*3508,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,11.742550,65.738760
2,5eu6,D,E,SGHTA,FQGTGA,ASSFIGGTDTQY,DSAIYN,IQSSQRE,AVLSSGGSNYKLT,C,...,TRBJ2-3*01,[human],GAGVSQTPSNKVTEKGKYVELRCDPISGHTALYWYRQSLGQGPEFL...,MH1,HLA-A*0201,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,36.412892,8.430048
3,7pbe,D,E,SEHNR,FQNEAQ,ASSSANSGELF,NSASQS,VYSSG,VVNINTDKLI,C,...,TRBJ2-2*01,[human],TGVSQDPRHKITKRGQNVTFRCDPISEHNRLYWYRQTLGQGPEFLT...,MH1,HLA-A*0201,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,2.091703,48.757504
4,8eo8,D,E,DFQATT,SNEGSKA,SAGPTSGRTDTQY,DSASNY,IRSNVGE,AADGGAGSYQLT,C,...,TRBJ2-3*01,[human],AVVSQHPSRVICKSGTSVKIECRSLDFQATTMFWYRQFPKQSLMLM...,MH1,HLA-B*3557,SHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTEP...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,7.896511,40.348448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,3gsn,A,B,MNHEY,SVGAGI,ASSPVTGGIYGYT,SSNFYA,MTLNGDE,ARNTGNQFY,P,...,TRBJ1-2*01,[human],MGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLIH...,MH1,HLA-A*0201,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,8.833760,36.271227
183,7ow6,D,E,MNHEY,SVGEGT,ASKVGPGQHNSPLH,TRDTAYY,QPWWGEQN,AMSVPSGDGSYQFT,C,...,TRBJ1-6*01,[human],NAGVTQTPKFRVLKTGQSMTLLCAQDMNHEYMYWYRQDPGMGLRLI...,MH1,HLA-A*1101,GSHSMRYFYTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,5.843942,46.599387
184,4jfe,D,E,GTSNPN,WGPFG,AWSETGLGMGGWQ,FLGSQS,TYREGD,AVNDGGRLT,C,...,TRBJ2-2*01,[human],SQTIHQWPATLVQPVGSPLSLECTVEGTSNPNLYWYRQAAGRGPQL...,MH1,HLA-A*0201,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,0.057799,41.074324
185,5men,D,E,MNHEY,SVGAGI,ASSYQGTEAF,DSVNN,IPSGT,AVDSATSGTYKYI,C,...,TRBJ1-1*01,[human],AGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLIH...,MH1,HLA-A*0201,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,"(human, B2M*02)",TPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKV...,7.392012,37.562161


In [12]:
selected_structures_summary.to_csv(os.path.join(output_dir, 'structures_summary.csv'), index=False)